In [8]:
!python -m pip install amazon-textract-caller --upgrade
!python -m pip install amazon-textract-response-parser --upgrade

  Obtaining dependency information for amazon-textract-response-parser from https://files.pythonhosted.org/packages/8f/9c/3e8fabe2c546aa3f8d052ef0ad3c425de184d7876ecabb38d87ed836378c/amazon_textract_response_parser-1.0.2-py2.py3-none-any.whl.metadata
  Attempting uninstall: amazon-textract-response-parser
    Found existing installation: amazon-textract-response-parser 1.0.1
    Uninstalling amazon-textract-response-parser-1.0.1:
      Successfully uninstalled amazon-textract-response-parser-1.0.1


In [9]:
import boto3
from IPython.display import Image, display
from trp import Document
from PIL import Image as PImage, ImageDraw
import time
from IPython.display import IFrame

In [10]:
# Curent AWS Region. Use this to choose corresponding S3 bucket with sample content

mySession = boto3.session.Session()
awsRegion = mySession.region_name

In [11]:
# S3 bucket that contains sample documents

s3BucketName = "s3BucketName_test"

In [12]:
# Amazon S3 client
s3 = boto3.client('s3')

# Amazon Textract client
textract = boto3.client('textract')

In [13]:
documentName = "input_1140169888_2023-10-2417:27:02.pdf"

In [14]:
# Step 1: Call Amazon Textract
response = textract.analyze_document(
    Document={
        'S3Object': {
            'Bucket': s3BucketName,
            'Name': documentName
        }
    },
    FeatureTypes=["TABLES", "FORMS"])

In [1]:
# Step 2: Create Amazon Textract Results Parser 
doc = Document(response)
for page in doc.pages:

    # Print lines and words
    for line in page.lines:
        print("Line: {}".format(line.text))
#         if identifier in line.text:
#             print(f"This pdf is for identifier: {identifier}")
            
    # Print forms fields
    for field in page.form.fields:
        key_text = field.key.text if field.key and field.key.text else "NULL"
        value_text = field.value.text if field.value and field.value.text else "NULL"
#         print("Field: Key: {}, Value: {}".format(key_text, value_text))

     # Print tables
    for table in page.tables:
        for r, row in enumerate(table.rows):
            for c, cell in enumerate(row.cells):
                print("Table[{}][{}] = {}".format(r, c, cell.text))

SyntaxError: incomplete input (1588495340.py, line 21)

In [2]:
# Step 3: Fetch fields: Name(first/last); BOD; Address; $ to be paid; NPI
# For Identifier: "Advanced Asthma And Allergy of Nny"

identifier = "identifier_test"
keys = ["NAME:", "DR NAME:", "NPI NO:"]
address_lines_idx = [9, 10]
coordinate_total_patient_resp = [7, 7]
fields = {}
for page in doc.pages:

    # Print lines and words
    count = 0
    address = ""
    for line in page.lines:
        if identifier in line.text:
            print(f"This pdf is for identifier: {identifier}")
        if count in address_lines_idx:
            address += line.text
            address += " "
        count += 1
    print(address)
    fields["ADDRESS:"] = address
            
    # Get field by key
    for key in keys:
        field = page.form.getFieldByKey(key)
        if(field):
            print("Field: Key: {}, Value: {}".format(field.key, field.value))      
            fields[key] = field.value.text

    for table in page.tables:
        for r, row in enumerate(table.rows):
            for c, cell in enumerate(row.cells):
#                 print("Table[{}][{}] = {}".format(r, c, cell.text))
                if r == coordinate_total_patient_resp[0] and c == coordinate_total_patient_resp[1]:
                    fields["Total Patient Responsibility:"] = "$" + cell.text
# print(fields)

NameError: name 'doc' is not defined

In [18]:
# Step 4: Using json.dump() method to write data to a JSON file
import json

with open('Output_1140169888_2023-10-2417:27:02.json', 'w') as json_file:
    json.dump(fields, json_file)